## Predicting Churn (True/False) using Deep Learning

In [19]:
#Dependencies 
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
seed = 1

In [20]:
#read data 


df = pd.read_csv('../Week-1/bigml.csv')
df.head()


,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [21]:
#Feature Scaling 

def normalize(df):
    for column in df.columns:

            if df[column].dtype not in ['object', 'bool']:
                max_ = df[column].max()
                min_ = df[column].min()
                df[column]= (df[column] - min_) / (max_ - min_)
    return df

df = normalize(df)
df.head()



,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,0.524793,0.068627,382-4657,no,yes,0.490196,0.755701,0.666667,0.755701,...,0.582353,0.542866,0.595750,0.408451,0.595935,0.500,0.15,0.500000,0.111111,False
1,OH,0.438017,0.068627,371-7191,no,yes,0.509804,0.460661,0.745455,0.460597,...,0.605882,0.537690,0.621840,0.492958,0.622236,0.685,0.15,0.685185,0.111111,False
2,NJ,0.561983,0.068627,358-1921,no,no,0.000000,0.693843,0.690909,0.693830,...,0.647059,0.333225,0.374933,0.500000,0.375374,0.610,0.25,0.609259,0.000000,False
3,OH,0.342975,0.000000,375-9999,yes,no,0.000000,0.853478,0.430303,0.853454,...,0.517647,0.170171,0.467187,0.394366,0.467424,0.330,0.35,0.329630,0.222222,False
4,OK,0.305785,0.068627,330-6626,yes,no,0.000000,0.475200,0.684848,0.475184,...,0.717647,0.407959,0.440290,0.619718,0.440526,0.505,0.15,0.505556,0.333333,False


In [22]:
#Drop Data and Split into Features and Label

df1 = df.drop(['state','area code','phone number'], axis=1)
x = df1.drop(['churn'], axis=1).values
y = df1['churn'].values


In [23]:
#Label Encoding categorical values

from sklearn.preprocessing import LabelEncoder 

x_labelEncoder = LabelEncoder()
x[:,1] = x_labelEncoder.fit_transform(x[:,1])
x[:,2] = x_labelEncoder.fit_transform(x[:,2])

y_labelEncoder = LabelEncoder()
y = y_labelEncoder.fit_transform(y)

In [13]:
x.shape

(3333, 17)

In [24]:
y.shape


(3333,)

In [25]:
from keras.utils import to_categorical
y_cat = to_categorical(y)

In [26]:
y_cat.shape


(3333, 2)

In [27]:
#Split data into test and train sets
x_train, x_test, y_train, y_test = train_test_split(x, y_cat,
                                                    random_state=22,
                                                    test_size=0.2)

In [28]:
#Build a Neural Network model 

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [33]:
model = Sequential()
model.add(Dense(32, input_shape=(17,), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.compile(Adam(lr=0.1),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 32)                576       
_________________________________________________________________
dense_34 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 66        
Total params: 1,698
Trainable params: 1,698
Non-trainable params: 0
_________________________________________________________________


In [35]:
#Fit model to train set 
model.fit(x_train, y_train, epochs=200, verbose=2, validation_split=0.1)

Train on 2399 samples, validate on 267 samples
Epoch 1/200
 - 1s - loss: 0.3929 - acc: 0.8520 - val_loss: 0.4487 - val_acc: 0.7640
Epoch 2/200
 - 0s - loss: 0.3552 - acc: 0.8579 - val_loss: 0.4088 - val_acc: 0.8277
Epoch 3/200
 - 0s - loss: 0.3602 - acc: 0.8591 - val_loss: 0.4129 - val_acc: 0.8277
Epoch 4/200
 - 0s - loss: 0.3400 - acc: 0.8595 - val_loss: 0.4057 - val_acc: 0.8277
Epoch 5/200
 - 0s - loss: 0.3507 - acc: 0.8595 - val_loss: 0.4077 - val_acc: 0.8277
Epoch 6/200
 - 0s - loss: 0.3417 - acc: 0.8595 - val_loss: 0.4231 - val_acc: 0.8277
Epoch 7/200
 - 0s - loss: 0.3465 - acc: 0.8583 - val_loss: 0.4119 - val_acc: 0.7790
Epoch 8/200
 - 0s - loss: 0.3367 - acc: 0.8554 - val_loss: 0.4062 - val_acc: 0.8277
Epoch 9/200
 - 0s - loss: 0.3267 - acc: 0.8595 - val_loss: 0.4478 - val_acc: 0.8277
Epoch 10/200
 - 0s - loss: 0.3278 - acc: 0.8595 - val_loss: 0.4057 - val_acc: 0.8277
Epoch 11/200
 - 0s - loss: 0.3288 - acc: 0.8595 - val_loss: 0.4015 - val_acc: 0.8277
Epoch 12/200
 - 0s - loss: 

Epoch 97/200
 - 0s - loss: 0.3274 - acc: 0.8541 - val_loss: 0.3880 - val_acc: 0.8277
Epoch 98/200
 - 0s - loss: 0.3289 - acc: 0.8541 - val_loss: 0.3900 - val_acc: 0.8277
Epoch 99/200
 - 0s - loss: 0.3185 - acc: 0.8595 - val_loss: 0.4056 - val_acc: 0.8277
Epoch 100/200
 - 0s - loss: 0.3265 - acc: 0.8595 - val_loss: 0.3880 - val_acc: 0.8277
Epoch 101/200
 - 0s - loss: 0.3235 - acc: 0.8545 - val_loss: 0.4733 - val_acc: 0.8277
Epoch 102/200
 - 0s - loss: 0.3198 - acc: 0.8595 - val_loss: 0.5011 - val_acc: 0.8277
Epoch 103/200
 - 0s - loss: 0.3294 - acc: 0.8595 - val_loss: 0.3978 - val_acc: 0.8277
Epoch 104/200
 - 0s - loss: 0.3213 - acc: 0.8591 - val_loss: 0.4132 - val_acc: 0.7753
Epoch 105/200
 - 0s - loss: 0.3309 - acc: 0.8549 - val_loss: 0.4130 - val_acc: 0.8277
Epoch 106/200
 - 0s - loss: 0.3200 - acc: 0.8595 - val_loss: 0.3995 - val_acc: 0.8277
Epoch 107/200
 - 0s - loss: 0.3183 - acc: 0.8595 - val_loss: 0.3894 - val_acc: 0.8277
Epoch 108/200
 - 0s - loss: 0.3219 - acc: 0.8595 - val_lo

Epoch 193/200
 - 0s - loss: 0.3310 - acc: 0.8533 - val_loss: 0.4153 - val_acc: 0.7828
Epoch 194/200
 - 0s - loss: 0.3220 - acc: 0.8599 - val_loss: 0.3929 - val_acc: 0.8277
Epoch 195/200
 - 0s - loss: 0.3188 - acc: 0.8595 - val_loss: 0.3831 - val_acc: 0.8277
Epoch 196/200
 - 0s - loss: 0.3258 - acc: 0.8595 - val_loss: 0.3865 - val_acc: 0.8277
Epoch 197/200
 - 0s - loss: 0.3235 - acc: 0.8595 - val_loss: 0.3898 - val_acc: 0.8277
Epoch 198/200
 - 0s - loss: 0.3244 - acc: 0.8595 - val_loss: 0.4107 - val_acc: 0.8277
Epoch 199/200
 - 0s - loss: 0.3206 - acc: 0.8529 - val_loss: 0.3856 - val_acc: 0.8277
Epoch 200/200
 - 0s - loss: 0.3149 - acc: 0.8595 - val_loss: 0.4093 - val_acc: 0.8277


In [37]:
#Evaluate model against test set
score = model.evaluate(x_test,y_test)

667/667 [==============================] - 0s 28us/step


In [38]:
#predict the values for x_test
pred = model.predict(x_test)

In [39]:
#Converting y into data whose accuracy can be measured
y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(pred, axis=1)

In [46]:
#Find accuracy
acc = accuracy_score(y_test_class,y_pred_class)
print("The accuracy of the NN Model is:", acc*100,"%")

The accuracy of the NN Model is: 85.00749625187406 %
